In [3]:
# LSTM for sequence classification in the IMDB dataset
from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras.layers.embeddings import Embedding
from tensorflow.keras.layers import Embedding, LSTM, Dense
from keras.layers import Dropout
from keras.preprocessing import sequence
import pandas as pd
import numpy as np
import math
import tensorflow as tf

In [4]:
SCALE = int("FFFFFFF",16)
# max_num = int(int("FFFFFF",16)/int("FFFFF",16)+1)
max_num = int(int("FFFFFF",16)/int("FFFFF",16)+1)
# max_num = 2
SEGMENT = 8

In [ ]:
def tokenizer_payload(hex_string, token_length = 4):
    # token_length = 4 
    # new token length according to the max tokens for the embedding; which is original 6   
    regex_pattern = '.{1,' + str(token_length) + '}'
    return ' '.join(re.findall(regex_pattern, hex_string))

In [ ]:
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191
top_n = 500 # take first 10 packets for test
filename = os.path.basename(csv_file)

df = pd.read_csv(csv_file)
df.dropna(subset = ['tcp.payload'], inplace=True)

df['tokenizer_content'] = df['tcp.payload'].apply(tokenizer_payload)
# encoding = tiktoken.get_encoding(embedding_encoding)

df["n_tokens"] = df.tokenizer_content.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
# df = df[df.n_tokens <= max_tokens]

In [17]:
def process_normal_data(filename):
    data = pd.read_csv(filename)
#     new_data = data.drop([0], axis = 0)
    new_data = data
    new_data = new_data.drop(new_data[pd.isna(new_data['tcp.payload'])].index)
    new_data.drop(new_data.head(20).index, inplace = True)
    new_data.to_csv('temp.csv',index = False)
    payload = pd.read_csv('temp.csv')
    
    X_set = []
    for i in range(len(payload['tcp.payload'])):
        # print(i)
        sample = payload['tcp.payload'][i]
        # print(sample)
        j = 0
        sample_list = []
        while j < len(sample):
            sample_list.append(int(sample[j:j+SEGMENT],16)/float(SCALE))
            j = j+SEGMENT
        if j>len(sample):
            sample_list.append(int(sample[j-SEGMENT:],16)/float(SCALE))
        
        X_set.append(np.array(sample_list))
    y_set = [0]*len(X_set)
    print(type(X_set))
    print(X_set[0])
    return np.array(X_set),np.array(y_set)

In [18]:
def process_attack_data(filename):
    data = pd.read_csv(filename)
#     new_data = data.drop([0], axis = 0)
    new_data = data
    new_data = new_data.drop(new_data[pd.isna(new_data['tcp.payload'])].index)
#     if filename == "../pcap file/attack file/csv_file/attack_DDoS.csv":     
#         new_data.to_csv('sequence.csv',index = False)
    new_data.drop(new_data.head(20).index, inplace = True)
    new_data.to_csv('temp.csv',index = False)
    payload = pd.read_csv('temp.csv')
    
    X_set = []
    for i in range(len(payload['tcp.payload'])):
        # print(i)
        sample = payload['tcp.payload'][i]
        # print(sample)
        j = 0
        sample_list = []
        while j < len(sample):
            sample_list.append(int(sample[j:j+SEGMENT],16)/float(SCALE))
            j = j+SEGMENT
        if j>len(sample):
            sample_list.append(int(sample[j-SEGMENT:],16)/float(SCALE))
        X_set.append(sample_list)

    y_set = [1]*len(X_set)
    print(type(X_set))
    print(X_set[0])
    return X_set,y_set

In [19]:
def train_test_split(X,y,ratio):
    if(len(X)!=len(y)):
        return false
    divide = math.ceil(len(X)*ratio)
    X_train = X[:divide]
    X_test = X[divide:]
    y_train = y[:divide]
    y_test = y[divide:]
    return X_train,X_test,y_train,y_test

In [20]:
prefix = "./test_payload/"
X_text_1,y_text_1 = process_normal_data(prefix + "3txt_sample10_trans_enc_utf8.csv")
X_attack_load_1,y_attack_load_1 = process_attack_data(prefix + "7attacked_932_loading.csv")

<class 'list'>
[6.18714347e+00 1.79802998e+00 1.15400727e+01 1.54010622e+01
 7.12486495e+00 8.68088516e+00 1.44769003e+01 1.50589067e+01
 5.71513505e+00 1.03861038e+01 1.41187771e+01 1.10396274e+01
 4.18805705e-03 7.42852371e+00 7.40260671e-01 5.72084279e+00
 3.74007766e+00 4.12769882e-01 8.95142521e+00 1.00958337e+01
 1.37748540e+01 5.22551878e-01 5.08022117e+00 3.44219633e+00
 5.23444979e+00 4.33722104e+00 6.33538437e+00 1.27822948e+01
 1.10705133e+01 1.24992761e+01 7.75199948e+00 1.82050422e+00
 4.52659999e+00 8.78751542e+00 8.40541919e+00 6.79420298e+00
 1.31443051e+01 1.38109577e+01 4.98816571e+00 3.95092185e+00
 1.36771452e+01 1.38211011e+01 7.21211790e+00 4.75836386e+00
 1.49843306e+01 1.32903723e+01 5.89519796e+00 1.42550933e+01
 6.29158417e+00 1.55342138e+01 1.39905151e+00 4.98586854e+00
 1.06019417e+01 7.84405766e+00 1.40085550e+01 1.43155616e+01
 1.01817910e+01 5.82852583e+00 2.67519799e+00 8.33055644e+00
 1.14677186e+01 4.18959739e+00 4.08615363e+00 5.59847858e+00
 4.091731

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (23899,) + inhomogeneous part.

In [ ]:
# prefix = "./pre_content_1/"

# #encrypted text
# X_text_1,y_text_1 = process_normal_data(prefix + "1txt_sample5clean_enc_ip159.csv")
# X_text_2,y_text_2 = process_normal_data(prefix + "2txt_sample9_trans_enc_utf8.csv")
# X_text_3,y_text_3 = process_normal_data(prefix + "3txt_sample10_trans_enc_utf8.csv")

In [ ]:
# # process attack data
# X_attack_load_1,y_attack_load_1 = process_attack_data(prefix + "7attacked_932_loading.csv")
# X_attack_load_2,y_attack_load_2 = process_attack_data(prefix + "8webcam932_novideo_load_ip12.csv")
# X_attack_load_3,y_attack_load_3 = process_attack_data(prefix + "9webcam5020_novideo_load_ip134.csv")

In [ ]:
# concoct the train set
X_train = np.row_stack((X_text_1,X_attack_load_1))
y_train = np.concatenate((y_text_1, y_attack_load_1),axis = 0)

In [ ]:
# model with dropout
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(max_num, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
# model.add(LSTM(100,return_sequences=True))
# model.add(LSTM(100,return_sequences=True))
# model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=64,shuffle=True)